In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json

In [2]:
session = requests.session()

# 1. Сбор стихов с сайта РуСтих

In [3]:
response = session.get('https://rustih.ru')

In [4]:
data = {}

stihi = BeautifulSoup(response.text, 'html.parser').find_all('div', {'class': 'post-card-one__content'})    # Получаем список стихотворений и ссылок, которые к ним ведут
for stih in stihi:
    link = stih.find('a')['href']
    name = link[18:]

    poem_text = BeautifulSoup(session.get(link).text, 'html.parser').find_all('div', {'class': 'entry-content poem-text'})  # Получаем текст произведения
    text = poem_text[0].text
    if 'Анализ' in text:            # убираем текст с анализом, если он есть
        i = text.index('Анализ')
        text = text[:i]

    poem_rat = BeautifulSoup(session.get(link).text, 'html.parser').find_all('div', {'class': 'entry-rating'})  # Получаем оценку, которую пользователи дали этому стихотворению
    rating = poem_rat[0].find_all('strong')[1].text
    rating = float(rating.replace(',', '.'))

    data[name[:-1]] = {'rating': rating, 'text': text}  # Записываем данные в словарь

In [5]:
# Проходимся по следующим страницам
for ind in range(2, 10):
    response = session.get('https://rustih.ru/page/' + str(ind) + '/')
    stihi = BeautifulSoup(response.text, 'html.parser').find_all('div', {'class': 'post-card-one__content'})
    for stih in stihi:
        link = stih.find('a')['href']
        name = link[18:]

        poem_rat = BeautifulSoup(session.get(link).text, 'html.parser').find_all('div', {'class': 'entry-rating'})
        if len(poem_rat[0].find_all('strong')) > 1:
            rating = poem_rat[0].find_all('strong')[1].text
            rating = float(rating.replace(',', '.'))
        
            poem_text = BeautifulSoup(session.get(link).text, 'html.parser').find_all('div', {'class': 'entry-content poem-text'})
            text = poem_text[0].text
            if 'Анализ' in text:
                i = text.index('Анализ')
                text = text[:i]

            data[name[:-1]] = {'rating': rating, 'text': text}

# 2. Сбор стихов с сайта ТыПоэт

На этом сайте стихи находятся в разных темах, поэтому нужно пройтись именно по разделам

In [7]:
# Раздел Оценка стихов
for ind in range(1, 43):
    stihi = BeautifulSoup(session.get('https://tipoet.com/list.php?type=ocenka&cat=1&page=' + str(ind)).text, 'html.parser').find_all('td', {'style': 'text-align: right; padding-right: 30px;'})
    for stih in stihi:
        link = stih.find('a')['href']
        name = link[len(link)-5:len(link)]

        if name != '11341' and name != '11234': # С этими стихотворениями возникает проблема парсинга 
            poem_text = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('table', {'align': 'center'})
            text = poem_text[2].text
            l = text.index('Оценка стихов')
            r = text.index('\n1\n')
            text = text[l+14:r]

            poem_rat = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('li', {'class': 'current-rating'})
            l = str(poem_rat[0]).index('width')
            r = str(poem_rat[0]).index('><')
            rating = float(str(poem_rat)[l+7:r-2])

            if name == '11233': # Дублирующееся стихотворение, так как у двух записей разные оценки, мы присваиваем среднюю между ними
                data[name] = {'rating': 4.1, 'text': text}
            else:
                if rating > 0:
                    data[name] = {'rating': 5 * 0.01 * rating, 'text': text}    # На этом сайте оценка отображается в процентах от оценки 5, поэтому нужно перевести проценты в численную оценку

У нас не хватало стихотворений с низкими оценками, поэтому далее собираются только стихотворения с оценкой меньше 4

In [81]:
# Категория Грустные стихи
for ind in range(1, 97):
    stihi = BeautifulSoup(session.get('https://tipoet.com/list.php?cat=8&page=' + str(ind)).text, 'html.parser').find_all('td', {'style': 'text-align: right; padding-right: 30px;'})
    for stih in stihi:
        link = stih.find('a')['href']
        name = link[len(link)-5:len(link)]

        poem_text = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('table', {'align': 'center'})
        text = poem_text[2].text
        l = text.index('Грустные стихи')
        r = text.index('\n1\n')
        text = text[l+15:r]

        poem_rat = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('li', {'class': 'current-rating'})
        l = str(poem_rat[0]).index('width')
        r = str(poem_rat[0]).index('><')
        rating = float(str(poem_rat)[l+7:r-2])

        if rating < 80 and rating > 0:
            data[name] = {'rating': 5 * 0.01 * rating, 'text': text}

In [82]:
# Категория История любви в стихах
for ind in range(1, 24):
    stihi = BeautifulSoup(session.get('https://tipoet.com/list.php?cat=9&page=' + str(ind)).text, 'html.parser').find_all('td', {'style': 'text-align: right; padding-right: 30px;'})
    for stih in stihi:
        link = stih.find('a')['href']
        name = link[len(link)-5:len(link)]

        poem_text = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('table', {'align': 'center'})
        text = poem_text[2].text
        l = text.index('История любви в стихах')
        r = text.index('\n1\n')
        text = text[l+23:r]

        poem_rat = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('li', {'class': 'current-rating'})
        l = str(poem_rat[0]).index('width')
        r = str(poem_rat[0]).index('><')
        rating = float(str(poem_rat)[l+7:r-2])

        if rating < 80 and rating > 0:
            data[name] = {'rating': 5 * 0.01 * rating, 'text': text}

In [83]:
# Категория Красивые стихи
for ind in range(1, 69):
    stihi = BeautifulSoup(session.get('https://tipoet.com/list.php?cat=5&page=' + str(ind)).text, 'html.parser').find_all('td', {'style': 'text-align: right; padding-right: 30px;'})
    for stih in stihi:
        link = stih.find('a')['href']
        name = link[len(link)-5:len(link)]

        if name != '15487':
            poem_text = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('table', {'align': 'center'})
            text = poem_text[2].text
            l = text.index('Красивые стихи')
            r = text.index('\n1\n')
            text = text[l+15:r]

            poem_rat = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('li', {'class': 'current-rating'})
            l = str(poem_rat[0]).index('width')
            r = str(poem_rat[0]).index('><')
            rating = float(str(poem_rat)[l+7:r-2])

            if rating < 80 and rating > 0:
                data[name] = {'rating': 5 * 0.01 * rating, 'text': text}

In [86]:
# Категория Любовные стихи
for ind in range(1, 69):
    stihi = BeautifulSoup(session.get('https://tipoet.com/list.php?cat=50&page=' + str(ind)).text, 'html.parser').find_all('td', {'style': 'text-align: right; padding-right: 30px;'})
    for stih in stihi:
        link = stih.find('a')['href']
        name = link[len(link)-5:len(link)]

        poem_text = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('table', {'align': 'center'})
        text = poem_text[2].text
        l = text.index('Любовные стихи')
        r = text.index('\n1\n')
        text = text[l+15:r]

        poem_rat = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('li', {'class': 'current-rating'})
        l = str(poem_rat[0]).index('width')
        r = str(poem_rat[0]).index('><')
        rating = float(str(poem_rat)[l+7:r-2])

        if rating < 80 and rating > 0:
            data[name] = {'rating': 5 * 0.01 * rating, 'text': text}

In [87]:
# Категория Печальные стихи
for ind in range(1, 33):
    stihi = BeautifulSoup(session.get('https://tipoet.com/list.php?cat=4&page=' + str(ind)).text, 'html.parser').find_all('td', {'style': 'text-align: right; padding-right: 30px;'})
    for stih in stihi:
        link = stih.find('a')['href']
        name = link[len(link)-5:len(link)]

        poem_text = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('table', {'align': 'center'})
        text = poem_text[2].text
        l = text.index('Печальные стихи')
        r = text.index('\n1\n')
        text = text[l+16:r]

        poem_rat = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('li', {'class': 'current-rating'})
        l = str(poem_rat[0]).index('width')
        r = str(poem_rat[0]).index('><')
        rating = float(str(poem_rat)[l+7:r-2])

        if rating < 80 and rating > 0:
            data[name] = {'rating': 5 * 0.01 * rating, 'text': text}

In [88]:
# Категория Философские стихи
for ind in range(1, 55):
    stihi = BeautifulSoup(session.get('https://tipoet.com/list.php?cat=54&page=' + str(ind)).text, 'html.parser').find_all('td', {'style': 'text-align: right; padding-right: 30px;'})
    for stih in stihi:
        link = stih.find('a')['href']
        name = link[len(link)-5:len(link)]

        poem_text = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('table', {'align': 'center'})
        text = poem_text[2].text
        l = text.index('Философские стихи')
        r = text.index('\n1\n')
        text = text[l+17:r]

        poem_rat = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('li', {'class': 'current-rating'})
        l = str(poem_rat[0]).index('width')
        r = str(poem_rat[0]).index('><')
        rating = float(str(poem_rat)[l+7:r-2])

        if rating < 80 and rating > 0:
            data[name] = {'rating': 5 * 0.01 * rating, 'text': text}

In [89]:
# Категория Чувственные стихи
for ind in range(1, 46):
    stihi = BeautifulSoup(session.get('https://tipoet.com/list.php?cat=10&page=' + str(ind)).text, 'html.parser').find_all('td', {'style': 'text-align: right; padding-right: 30px;'})
    for stih in stihi:
        link = stih.find('a')['href']
        name = link[len(link)-5:len(link)]

        poem_text = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('table', {'align': 'center'})
        text = poem_text[2].text
        l = text.index('Чувственные стихи')
        r = text.index('\n1\n')
        text = text[l+18:r]

        poem_rat = BeautifulSoup(session.get('https://tipoet.com' + link).text, 'html.parser').find_all('li', {'class': 'current-rating'})
        l = str(poem_rat[0]).index('width')
        r = str(poem_rat[0]).index('><')
        rating = float(str(poem_rat)[l+7:r-2])

        if rating < 80 and rating > 0:
            data[name] = {'rating': 5 * 0.01 * rating, 'text': text}

# 3. Запись данных в файл

In [91]:
with open('data.json', 'w', encoding='utf8') as file:
    json.dump(data, file, ensure_ascii=False)